In [71]:
!git add Parser.ipynb
!git commit -m "finished parsing part"
!git push

[master cee175c] finished parsing part
 1 file changed, 648 insertions(+), 616 deletions(-)
Counting objects: 5, done.
Delta compression using up to 4 threads.
Compressing objects: 100% (4/4), done.
Writing objects: 100% (5/5), 6.10 KiB | 0 bytes/s, done.
Total 5 (delta 2), reused 0 (delta 0)
remote: Resolving deltas: 100% (2/2), completed with 2 local objects.
To https://github.com/DmitrySerg/OpenData
   459dc13..cee175c  master -> master


## Цель
- Собрать базу открытых данных с сайта Московской патриархии о персоналиях, организациях, богослужебных указаниях и храмах, структурировать их и загрузить в хаб открытых данных

## Задача
- проинвентаризировать данные на сайте Московской патриархии
- написать парсеры и выгрузить данные в форматах CSV и JSON
- загрузить данные в хаб открытых данных http://hubofdata.ru в удобном виде.
## Требования
- открытый исходный код в Github под свободной лицензией
- межплатформенный код (возможность запуска на Linux/Windows)
- предусмотреть перенос всех метаданных в CKAN в виде тегов и атрибутов
- переносить все данные в CKAN или на отдельный хостинг (например, в Github)
## Данные
- Персоналии - http://www.patriarchia.ru/db/persons/
- Организации - http://www.patriarchia.ru/db/organizations/
- Богослужебные указания - http://www.patriarchia.ru/bu/
- Храмы - http://map.patriarhia.ru/
## Пожелания
- реализация в виде программы командной строки или веб-приложения
- возможность запуска на MacOSX
- использовать скриптовые языки такие как Python, Perl, R и другие.

In [69]:
import requests      
import numpy as np   
import pandas as pd  
import time          
from bs4 import BeautifulSoup
from joblib import Parallel, delayed

import os
import json
import string

#from tqdm import tqdm_notebook
from tqdm import tqdm

In [2]:
main_link = "http://www.patriarchia.ru"
link_persons = "http://www.patriarchia.ru/db/persons/"
link_organizations = "http://www.patriarchia.ru/db/organizations/"
link_guidance = "http://www.patriarchia.ru/bu/"
link_churches = "http://map.patriarhia.ru/"
link_map_objects = 'http://map.patriarhia.ru/api/index.php?token=c2cbe8cd873f8e4751a7f7bb5292e68a&s=&params=[]&_=1530217371426'

file_path = os.getcwd() + "/data/{}"

# Богослужебные указания доступны на следующие даты
start_date = '2009-01-14'
end_date = '2019-01-13'

In [55]:
class Parser:
    
    def __init__(self):
        pass
    
    def getSoup(self, link):
        """
            Returns bs4-formatted page
        """
        html = requests.get(link)
        if html.status_code == 200:
            soup = BeautifulSoup(html.content, 'lxml')
        else:
            soup = None
        time.sleep(0.5)
        return soup
    
    def getAllLinks(self, dataset_type, url=None):
        """
            Returns a list with names, links and affiliations for persons and organizations;
            dates for guidance or map objects

            dataset_type: string
                persons/organizations/guidance/map
                
            url: string
                url to parse data from
        """
        
        if dataset_type=='guidance':
            datelist = pd.date_range(
                pd.to_datetime(start_date), end=pd.to_datetime(end_date)
            ).tolist()
            datelist = [date.strftime("%Y-%m-%d") for date in datelist]
            return datelist
        
        if dataset_type=='map':
            map_objects = requests.get(link_map_objects)
            map_objects = map_objects.content
            map_objects = json.loads(map_objects)['features']
            return map_objects

        links = self.getSoup(url)
        links = links.find('div', attrs={"class":"main"})
        links = links.findAll('div', attrs={'class':'news'})

        dfLinks = []
        for link in links:
            name = link.find('h4').text
            linkParsed = main_link + link.find('a')['href']
            affiliation = link.find('div').text
            dfLinks.append({"name":name, "link":linkParsed, "affiliation":affiliation})

        return dfLinks

    def getPageData(self, link):
        """
            Returns a dict with parsed field:value pairs from page

            link: string
                link to page
        """

        data = {}
        try:
            # use printing version of page 
            if 'text' in link:
                link = link.replace("text", "print")

            # first - get the page content
            pageContent = getSoup(link)

            # trying to find any images
            try:
                image_link = pageContent.find('div', attrs={'class':'news_img'})
                image_link = image_link.find('img')['src']
                data.update({'image_link':image_link})
            except:
                pass

            # now concentrate on text
            pageContent = pageContent.find('div', attrs={'class':'main'})
            pageContent = pageContent.findAll("dl")

            # second - find all field-value pairs on page
            for text in pageContent:        
                number_of_fields = len(text.findAll('dt'))*2-1
                if number_of_fields>0:
                    parsedFields = []
                    currentValue = text.find('dt')
                    parsedFields.append(' '.join(
                        currentValue.findAll(text=True)).strip().replace(u'\xa0', u' '))

                    while number_of_fields>0:
                        currentValue = currentValue.nextSibling
                        if currentValue != '\n':
                            value = ' '.join(currentValue.findAll(text=True))
                            value = value.strip().replace(u'\xa0', u' ')
                            if value:
                                parsedFields.append(value)
                                number_of_fields -= 1

                    text = dict(zip(parsedFields[::2], parsedFields[1::2]))
                    data.update(text)

        except:
            pass
        return data
    
    def getGuidanceData(self, date):
        """
            Returns dict with name:text pair for particular day
        """
        try:
            link = link_guidance + date + '/print.html'
            pageContent = self.getSoup(link)
            pageContent = pageContent.find('div', attrs={'class':'main'})
            pageContent = pageContent.findAll('p')

            name = pageContent[0].getText().strip()
            pageContent = [paragraph.getText() for paragraph in pageContent[1:]]
            text = "\n".join(pageContent)
            data = {'date':date, 'name':name, 'text':text}
        except:
            data = {}
        return data
    
    def getMapObjectData(self, current_object):
        """
            Parse data of a map object, return dict with values

            current_object: bs4.element.Tag
                bs4 object from map
        """
        data = {}
        try:
            # parsing map info
            infoid = current_object['id']
            lat, long = current_object['geometry']['coordinates']
            name = current_object['properties']['hintContent']

            # moving to object page
            current_link = 'http://map.patriarhia.ru/?infoid={}'.format(infoid)
            data.update({"link":current_link, "lat":lat, "long":long, "name":name})
            current_object = self.getSoup(current_link)
            current_object = current_object.find('div', attrs={'class':'main'})

            # address
            address = current_object.find('dl', attrs={'class':'addr'})
            if address:
                data.update({'address':address.getText()})

            # contact info
            contacts = current_object.find("div", attrs={'class':"reg-info-contact"})
            contacts = contacts.findAll('dl')
            contacts = contacts[0].findAll('dd')
            for current_contact in contacts:
                contact_type = current_contact.find('span').get('class')[0]
                contact_value = current_contact.getText()
                data.update({contact_type:contact_value})

            # dependent structure info
            structures = current_object.find('div', attrs={'id':'parent_site_list'})
            if structures:
                structures = structures.findAll('h1')
                structures = ', '.join([structure.getText().strip() for structure in structures])
                data.update({'structure':structures})
        except:
            pass
        
        return data
    
    def saveDataFrame(self, final, dataset_type):
        """
            Save final file into csv and json files
            
            final: list
                parsed data from pages
                
            dataset_type: string
                persons or organizations
        """
        
        df = pd.DataFrame(final)
        df.columns = df.columns.str.strip(':')
        
        if dataset_type=="persons":
            csv_path = file_path.format("persons.csv")
            json_path = file_path.format("persons.json")
            column_names = ['name','affiliation','Страна','Дата рождения', 
                           'Дата пострига','Дата хиротонии', 'Дата смерти',
                           'День ангела', 'Епархия', 'Место работы', 'Образование', 
                           'Биография', 'Награды', 'Научные труды, публикации', 
                           'E-mail', 'Web-сайт', 'link', 'image_link']
            df = df[column_names]
        elif dataset_type=="organizations":
            csv_path = file_path.format("organizations.csv")
            json_path = file_path.format("organizations.json")
        elif dataset_type=="map":
            csv_path = file_path.format("map_objects.csv")
            json_path = file_path.format("map_objects.json")
        else:
            csv_path = file_path.format("guidance.csv")
            json_path = file_path.format("guidance.json")
        
        df.to_csv(csv_path, encoding='utf8')
        df.to_json(json_path)
            
            
    def runCrawler(self, links_data, dataset_type='other', run_in_parallel=False, n_jobs=1):
        """
            Iterate over all links in links_data and return final dataset
            
            links_data: list
                list with links to pages
            
            dataset_type: string
                guidance/map or other
                
            run_in_parallel: bool
                if True - run parsing in multithreads using joblib
            
            n_jobs: int
                only if run_in_parallel is True - number of threads
        """
        final = []
        
        # running in parallel - using joblib functions
        if run_in_parallel:
            if dataset_type=='guidance':
                final = Parallel(n_jobs=n_jobs)(
                    delayed(self.getGuidanceData)(current_object) \
                    for current_object in tqdm_notebook(links_data))
            elif dataset_type=='map':
                final = Parallel(n_jobs=n_jobs)(
                    delayed(self.getMapObjectData)(current_object) \
                    for current_object in tqdm_notebook(links_data))
            else:
                final = Parallel(n_jobs=n_jobs)(
                    delayed(self.getPageData)(page['link']) \
                    for page in tqdm_notebook(links_data))
        # running with single thread 
        else:
            for page in tqdm_notebook(links_data):
                try:
                    if dataset_type=='guidance':
                        currentDateData = self.getGuidanceData(page)
                        final.append(currentDateData)
                    elif dataset_type=='map':
                        object_data = self.getMapObjectData(page)
                        final.append(object_data)
                    else:
                        data = self.getPageData(page['link'])
                        page.update(data)
                        final.append(page)
                    
                except Exception as e:
                    print(e)                 # DEBUG
                    print('problem',  page)  # DEBUG
                    time.sleep(30)
                
        return final
        
    
    def runParsing(self, dataset_type, run_in_parallel=False, n_jobs=1):
        
        if dataset_type=='persons':
            link = link_persons
        elif dataset_type=='organizations':
            link = link_organizations
        else:
            link = None
        
        allLinks = self.getAllLinks(dataset_type, link)
        data = self.runCrawler(allLinks, dataset_type, run_in_parallel, n_jobs)    
        
        self.saveDataFrame(data, dataset_type)

In [ ]:
if __name__ == "__main__":
    parser = Parser()
    parser.runParsing('persons')
    parser.runParsing('organizations')
    parser.runParsing('guidance', run_in_parallel=True, n_jobs=4)
    parser.runParsing('map', run_in_parallel=True, n_jobs=4)

In [67]:
persons = pd.read_csv('data/persons.csv', index_col=0)
persons.head()

,name,affiliation,Страна,Дата рождения,Дата пострига,Дата хиротонии,Дата смерти,День ангела,Епархия,Место работы,Образование,Биография,Награды,"Научные труды, публикации",E-mail,Web-сайт,link,image_link
0,"Августин, епископ Городецкий и Ветлужский (Ани...",Епископат РПЦ,Россия,14 января 1945 г.,5 сентября 1993 г.,8 апреля 2012 г.,NaN,NaN,Городецкая епархия \n (Правящий архиерей),NaN,1977 г. — Всесоюзный юридический заочный инсти...,"Родился 14 января 1945 г. в Москве, крещен в м...",NaN,Слово архимандрита Августина (Анисимова) при ...,gor.eparchia@gmail.com,www.egiv.ru,http://www.patriarchia.ru/db/text/2147951.html,http://p2.patriarchia.ru/2017/11/29/1236714685...
1,"Августин, митрополит Белоцерковский и Богуслав...",Епископат РПЦ,Украина,7 апреля 1952 г.,16 сентября 1992 г.,20 сентября 1992 г.,NaN,28 июня,Белоцерковская епархия \n (Правящий архиерей),NaN,1975 г. — Московская духовная семинария. 1982 ...,Родился 7 апреля 1952 г. в с. Глушковичи Лельч...,Церковные: 1998 г. — орден прп. Нестора Летопи...,NaN,NaN,www.bc-eparchy.org.ua,http://www.patriarchia.ru/db/text/52361.html,http://p2.patriarchia.ru/2013/12/06/1234619995...
2,"Агапит, архиепископ Штутгартский (РПЦЗ), викар...",Епископат РПЦ,Германия,29 сентября 1955 г.,29 марта 1983 г.,1 мая 2001 г.,NaN,3 марта,Берлинская и Германская епархия (РПЦЗ) \n (Вик...,NaN,NaN,Родился 29 сентября 1955 г. в Германии в чешск...,Церковные: 2015 г. — орден прп. Серафима Сар...,Слово при наречении во епископа Штутгартского...,agapit@rocor.de,www.rocor.de,http://www.patriarchia.ru/db/text/424847.html,NaN
3,"Агапит, митрополит Могилев-Подольский и Шаргор...",Епископат РПЦ,Украина,6 марта 1965 г.,1985 г.,22 ноября 1998 г.,NaN,14 июня,Могилев-Подольская епархия \n (Правящий архиерей),NaN,1994 г. — Киевская духовная семинария. 1998 г....,Родился 6 марта 1965 г. в с. Давыдовка Чернови...,Церковные: 2008 г. — орден прп. Сергия Радон...,Слово архимандрита Агапита (Бевцика) при наре...,NaN,NaN,http://www.patriarchia.ru/db/text/67725.html,http://p2.patriarchia.ru/2017/09/15/1234635359...
4,"Агафангел, епископ Норильский и Туруханский (Д...",Епископат РПЦ,Россия,30 сентября 1975 г.,17 января 1996 г.,6 июля 2014 г.,NaN,NaN,Норильская епархия \n (Правящий архиерей),NaN,2003 г. — Московская духовная семинария (заочн...,Родился 30 сентября 1975 г. в г. Норильске Кра...,NaN,Слово архимандрита Агафангела (Дайнеко) при н...,norilskeparhia@mail.ru,http://norilskeparhia.ru,http://www.patriarchia.ru/db/text/3667680.html,http://p2.patriarchia.ru/2014/07/25/1235350578...


In [66]:
guidance = pd.read_csv('data/guidance.csv', index_col=0)
guidance.head()

,date,name,text
0,2009-01-14,Среда. Обре́зание Господне. Свт. Василия Велик...,Примечание. В Типиконе сказано: «А́ще храм свя...
1,2009-01-15,Четверг. Предпразднство Богоявления. Прп. Сера...,"Свт. Сильве́стра, папы Римского. Прав. Иулиани..."
2,2009-01-16,Пятница. Предпразднство Богоявления. Прор. Мал...,Служба прор. Малахии и мч. Гордия не имеет пра...
3,NaN,NaN,NaN
4,2009-01-18,"Неделя 31-я по Пятидесятнице, пред Богоявление...",Прп. Синклитики́и Александрийской.\nВоскресная...


In [65]:
organizations = pd.read_csv('data/organizations.csv',  index_col=0)
organizations.head()

,E-mail,Web-сайт,affiliation,link,name,Администратор приходов,Адрес,Благочинный,Викарный архиерей,Временный управляющий,...,Секретарь Патриарха Московского и всея Руси по г. Москве,Секретарь,Сопредседатель,Страна,Телефон,Управляющий делами,Управляющий епархией,Управляющий приходами,Управляющий,Факс
0,abakan-eparchy@mail.ru,www.abakan-eparchy.ru,Епархии,http://www.patriarchia.ru/db/text/31086.html,Абаканская епархия,NaN,"655012, Россия, Республика Хакасия, г. Абакан,...",NaN,NaN,NaN,...,NaN,NaN,NaN,Россия,"(3902) 24-29-47 (канцелярия), 24-37-10 (секрет...",NaN,NaN,NaN,NaN,(3902) 24-29-15
1,adm@patriarchia.ru,NaN,Московская Патриархия,http://www.patriarchia.ru/db/text/961771.html,Административный секретариат Московской Патриа...,NaN,"115191, Москва, Даниловский вал, 22",NaN,NaN,NaN,...,NaN,NaN,NaN,Россия,(499) 578-03-58,NaN,NaN,NaN,NaN,NaN
2,alat-eparhia@mail.ru,http://alat-eparhia.ru,Епархии,http://www.patriarchia.ru/db/text/2511388.html,Алатырская епархия,NaN,"429820, Россия, Чувашская Республика, г. Алаты...",NaN,NaN,NaN,...,NaN,NaN,NaN,Россия,(83531) 2-53-06; 2-53-20 (канцелярия),NaN,NaN,NaN,NaN,NaN
3,NaN,http://orthodoxalbania.org,Поместные Церкви,http://www.patriarchia.ru/db/text/134409.html,Албанская Православная Церковь,NaN,Kisha Othodokse 151 Rruga e Kavejes Tirana — A...,NaN,NaN,NaN,...,NaN,NaN,NaN,Албания,NaN,NaN,NaN,NaN,NaN,NaN
4,eparhiyamaterials@gmail.com,http://alexandria-eparhia.org.ua,Епархии,http://www.patriarchia.ru/db/text/134627.html,Александрийская епархия,NaN,"28000, Украина, Кировоградская область, г. Але...",NaN,NaN,NaN,...,NaN,NaN,NaN,Украина,"(05235) 7-90-00, +380665488496 (кафедральный с...",NaN,NaN,NaN,NaN,NaN


In [64]:
map_objects = pd.read_csv('data/map_objects.csv', index_col=0)
map_objects.head()

,address,elitsy,email,fb,gplus,instagram,lat,link,long,name,ok,phone,site,skype,structure,twitter,vk,youtube
0,"Адрес:Россия, Мурманская область, г. Мурманск,...",NaN,oksana.artemenko@mail.ru,NaN,NaN,NaN,69.025935,http://map.patriarhia.ru/?infoid=32421,33.082167,Мурманская епархия,NaN,NaN,https://vk.com/hramvladimira,NaN,"Кандалакшское благочиние, Ковдорское благочини...",NaN,NaN,NaN
1,"Адрес:Украина, г. Киев, ул. Лаврская, 15, корп...",NaN,presschurch@gmail.com,NaN,NaN,NaN,50.432043,http://map.patriarhia.ru/?infoid=2738,30.561857,Украинская Православная Церковь,NaN,(10-380-44) 255-12-13,NaN,NaN,"Александрийская епархия, Балтская епархия, Бел...",NaN,NaN,NaN
2,"Адрес:Москва, Даниловский вал, 22, здание Отде...",NaN,NaN,NaN,NaN,NaN,55.710849,http://map.patriarhia.ru/?infoid=4231,37.627849,Зарубежные учреждения Московского Патриархата,NaN,NaN,NaN,NaN,Администрация Русской Православной Церкви в Ит...,NaN,NaN,NaN
3,"Адрес:214000, г. Смоленск, Соборный двор, 17",NaN,eparxysmolensk@mail.ru,NaN,NaN,NaN,54.782526,http://map.patriarhia.ru/?infoid=24076,32.044128,Смоленская митрополия,NaN,"+7 (4812) 38-01-97, факс: +7 4812 38-19-28",http://smoleparh.ru/,NaN,"Вяземская епархия, Рославльская епархия, Смоле...",NaN,NaN,NaN
4,"Адрес:г. Ставрополь, ул. Дзержинского, 157",NaN,krest@mail.stv.ru,https://www.facebook.com/stavropoleparhia/,NaN,NaN,45.044196,http://map.patriarhia.ru/?infoid=29675,41.961421,Ставропольская митрополия,NaN,+7 (8652) 35-51-63,http://stavropol-eparhia.ru,NaN,"Георгиевская епархия, Ставропольская епархия",NaN,NaN,NaN
